In [2]:
# Import libraries
import pandas as pd
import requests

# Import API keys
from alyssa_config import spoonacular_key, rapidapi_key

In [72]:
# Get list of recipe IDs
recipe_url = "https://api.spoonacular.com/recipes/complexSearch"
recipe_params = {
    'apiKey': spoonacular_key,
    'number': 100}

recipe_response = requests.get(recipe_url, recipe_params).json()
recipe_response

# Output recipe IDs to a csv
recipes_df = pd.DataFrame(recipe_response['results'])
recipes_df.to_csv('Resources/initial_recipes_2.csv', index=False)

In [73]:
# Import existing recipe IDs
recipes_df = pd.read_csv('Resources/initial_recipes_2.csv')

In [74]:
recipes_df.head()

,id,title,image,imageType
0,782585,Cannellini Bean and Asparagus Salad with Mushr...,https://spoonacular.com/recipeImages/782585-31...,jpg
1,716426,"Cauliflower, Brown Rice, and Vegetable Fried Rice",https://spoonacular.com/recipeImages/716426-31...,jpg
2,715497,Berry Banana Breakfast Smoothie,https://spoonacular.com/recipeImages/715497-31...,jpg
3,715415,Red Lentil Soup with Chicken and Turnips,https://spoonacular.com/recipeImages/715415-31...,jpg
4,716406,Asparagus and Pea Soup: Real Convenience Food,https://spoonacular.com/recipeImages/716406-31...,jpg


In [9]:
for id in recipes_df['id']:
    print(id)

782585
716426
715497
715415
716406
644387
715446
782601
795751
766453
716627
664147
640941
715495
794349
716381
660306
756814
715769
642129
642605
636589
646738
715540
663559
662670
640062
715421
648320
716432
798400
782600
652417
639535
729366
661925
794538
649931
636787
641975
652423
715447
716311
715543
157344
658509
716195
716361
765011
660228
716408
661340
782622
715424
659135
639851
664547
633921
658579
715385
641057
655575
632269
637162
982371
1095753
660405
715523
632252
639891
646651
716330
640318
642085
716437
664090
636602
716276
649944
646043
634437
655235
652393
640828
769774
715533
639411
636228
661259
664680
715391
636230
632812
633942
632347
975070
649886
659927
716217
636608


In [75]:
# Get recipe and nutrition information
nutrition_list = []
for id in recipes_df['id']:
    nutrition_url = f"https://api.spoonacular.com/recipes/{id}/information?"
    nutrition_params = {
        'apiKey': spoonacular_key,
        'includeNutrition': "true"}

    nutrition_response = requests.get(nutrition_url, nutrition_params).json()
    nutrition_list.append(nutrition_response)

In [82]:
nutrition_list[50]

{'status': 'failure',
 'code': 402,
 'message': 'Your daily points limit of 150 has been reached. Please upgrade your plan to continue using the API.'}

In [80]:
recipe_id = []
recipe_name = []
recipe_likes = []
meal_type = []
cuisines = []
num_ingredients = []

calories_amt = []
calories_unit = []
calories_pct = []

satfat_amt = []
satfat_unit = []
satfat_pct = []

sugar_amt = []
sugar_unit = []
sugar_pct = []

sodium_amt = []
sodium_unit = []
sodium_pct = []

protein_amt = []
protein_unit = []
protein_pct = []


for idx, row in enumerate(nutrition_list):
    try:
        recipe_id.append(nutrition_list[idx]['id'])
        recipe_name.append(nutrition_list[idx]['title'])
        recipe_likes.append(nutrition_list[idx]['aggregateLikes'])
        meal_type.append(nutrition_list[idx]['dishTypes'])
        cuisines.append(nutrition_list[idx]['cuisines'])
        num_ingredients.append(len(nutrition_list[idx]['extendedIngredients']))
        
        # Second to last '0' index is for CALORIES
        calories_amt.append(nutrition_list[idx]['nutrition']['nutrients'][0]['amount'])
        calories_unit.append(nutrition_list[idx]['nutrition']['nutrients'][0]['unit'])
        calories_pct.append(nutrition_list[idx]['nutrition']['nutrients'][0]['percentOfDailyNeeds'])
        
        satfat_amt.append(nutrition_list[idx]['nutrition']['nutrients'][2]['amount'])
        satfat_unit.append(nutrition_list[idx]['nutrition']['nutrients'][2]['unit'])
        satfat_pct.append(nutrition_list[idx]['nutrition']['nutrients'][2]['percentOfDailyNeeds'])
        
        sugar_amt.append(nutrition_list[idx]['nutrition']['nutrients'][5]['amount'])
        sugar_unit.append(nutrition_list[idx]['nutrition']['nutrients'][5]['unit'])
        sugar_pct.append(nutrition_list[idx]['nutrition']['nutrients'][5]['percentOfDailyNeeds'])
        
        sodium_amt.append(nutrition_list[idx]['nutrition']['nutrients'][7]['amount'])
        sodium_unit.append(nutrition_list[idx]['nutrition']['nutrients'][7]['unit'])
        sodium_pct.append(nutrition_list[idx]['nutrition']['nutrients'][7]['percentOfDailyNeeds'])
        
        protein_amt.append(nutrition_list[idx]['nutrition']['nutrients'][8]['amount'])
        protein_unit.append(nutrition_list[idx]['nutrition']['nutrients'][8]['unit'])
        protein_pct.append(nutrition_list[idx]['nutrition']['nutrients'][8]['percentOfDailyNeeds'])
    
    except:
        print(idx)

spoonacular_df = pd.DataFrame({
    'ID': recipe_id,
    'Name': recipe_name,
    'Likes': recipe_likes,
    'Meal Type': meal_type,
    'Cuisines': cuisines,
    'N_ingredients': num_ingredients,
    'Calories (Amount)': calories_amt,
    'Calories (Unit)': calories_unit,
    'Calories (% of Daily Needs)': calories_pct,
    'Saturated Fat (Amount)': satfat_amt,
    'Saturated Fat (Unit)': satfat_unit,
    'Saturated Fat (% of Daily Needs)': satfat_pct,
    'Sugar (Amount)': sugar_amt,
    'Sugar (Unit)': sugar_unit,
    'Sugar (% of Daily Needs)': sugar_pct,
    'Sodium (Amount)': sodium_amt,
    'Sodium (Unit)': sodium_unit,
    'Sodium (% of Daily Needs)': sodium_pct,
    'Protein (Amount)': protein_amt,
    'Protein (Unit)': protein_unit,
    'Protein (% of Daily Needs)': protein_pct})
unit_check = spoonacular_df['Protein (Unit)'].unique()
unit_check

spoonacular_df.to_csv("Resources/initial_nutrition_2.csv", index=False)

33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


782585